## Genetic

In Section 1: Definition of Concepts:
Input: Some target image.

Output: Our created image using genetic algorithms.
___

1. Each chromosome is generated as an image using our algorithm.
2. Each gene corresponds to a triangle, such that each chromosome is constructed as a composite of multiple genes (triangles).
3. Fitness functions are employed to evaluate the similarity between the current chromosome and the target image.
4. Mutation within each gene can occur through two distinct methods: one involves altering the position of a vertex within a triangle, while the other pertains to modifications in the triangle's color attributes.
5. Cross Over for 2 cromosome is slicing triangles into two parts and combine 2 subparts.

Just importing some libraries

In [26]:
import random
from PIL import Image, ImageDraw
import numpy as np
import cv2
import colour 
import copy
#from colour.difference import delta_E_CIE1976
def clamp(val, a, b):
   if(val < a):
      return a
   if(val > b):
      return b
   return val

Implementing the genes using Triangle class:

In [27]:
OFFSET = 3
def generate_point(width, height):
    x = random.randrange(0 - OFFSET, width + OFFSET, 1)
    y = random.randrange(0 - OFFSET, height + OFFSET, 1)
    return (x, y)
class Triangle:
    def __init__(self, img_width, img_height):
        self.points = []
        for i in range(3):
            self.points.append(generate_point(img_width,img_height))
        self.color = (
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
        )
        self._img_width = img_width
        self._img_height = img_height

Section 2: Initial Population Generation: 
In init function of chromosome we have made some random cromosome using random triangles. "self.triangles"
______________________________________________________
**Section 3: Fitness Evaluation:**

For each pixel in both the generated image and the target image, the fitness function calculates the color difference. This is achieved by computing with MSE method. Means for each pixel it calculates the sum of squares of color differences and then sqrt them.
Other method is by SSIM, its more powerfull than MSE but i prefered to use MSE cause of it's comfortable and easy implementation.
 The greater this difference becomes, the lower the fitness function should be, as it is an inverse relationship.

______________________________________________________
Section 4: Implementation of mutation, crossover, and next generation selection strategy:

**Mutation Strategy:**

The mutation can occur through one of two methods: the first involves altering the position of a point within the gene (triangle), while the second method entails changing the color attributes of the gene.

**Cross Over**
We have 2 cross over implemented, one point and 2 points and there are some probabilities to switch between them. Each of them gives 2 parent chromosomes and combine some continuous triangles parts with each other.

In [28]:
class Chromosome:
    def __init__(self,img_height, img_width,target_image,num_triangles):
        self.img_height = img_height
        self.img_width = img_width
        self.num_triangles = num_triangles
        self.triangles = [Triangle(img_width, img_height) for _ in range(num_triangles)]   # Create initial triangles
        self.target_image = target_image
        self.background_color = (0,0,0,255)
    def to_array(self, image):
            return np.array(image)
    def mutate(self, triangle):
        #location mutation
        for i in range(3):
          for j in range(2):
          
              newco= clamp(int( triangle.points[i][j] + (random.gauss(0,1)*20)), 0, max(self.img_height, self.img_width ))
              poi= triangle.points[i]
              if j==0: triangle.points[i]= (newco, poi[1])
              else: triangle.points[i]= (poi[0], newco)
        #color mutation
        col_to_update = int(random.random()*4) #only update one of the colours
        if(col_to_update == 0):#Red
          newco= clamp( int ( triangle.color[0] + random.gauss(0,1)*30), 0, 255)
          triangle.color = (newco, triangle.color[1], triangle.color[2], triangle.color[3])
        if(col_to_update == 1):#Green
          newco= clamp( int ( triangle.color[1] + random.gauss(0,1)*30), 0, 255)
          triangle.color = (triangle.color[0], newco,  triangle.color[2], triangle.color[3])
        if(col_to_update == 2):#Blue
          newco = clamp( int ( triangle.color[2] + random.gauss(0,1)*30), 0, 255)
          triangle.color = (triangle.color[0],  triangle.color[1], newco, triangle.color[3])
        if(col_to_update == 3): # alpha
          newco = clamp( int ( triangle.color[3] + random.gauss(0,1)*30), 10, 245)
          triangle.color = (triangle.color[0],  triangle.color[1], triangle.color[2], newco)
    def draw(self) -> Image:
        size = self.target_image.size
        img = Image.new('RGB', size, self.background_color)
        draw = ImageDraw.Draw(img)
        for triangle in self.triangles:
            points = triangle.points
            color = triangle.color
            draw.polygon(points, fill=color, outline=color)
        return img
    def fitness(self):# the differences between our image and input image
      target = np.array(self.target_image) / 255.0  
      created = np.array(self.draw()) / 255.0  
      mse = np.mean((created - target) ** 2)
      return -1*mse

      


Cross Over makes some split point with random function and implement one crossover method.
General mutation strategy is giving some parent and with probability of 50% mutate each triangle of itself.
In run function, initially we send the best cromosomes without changing to next generation...

In [53]:
class GeneticAlgorithm():
    def __init__(self,max_width,max_height,target_image, population_size, triangles_number):
        self.population_size = population_size
        self.max_width = max_width
        self.max_height = max_height
        self.population = [Chromosome(max_height,max_width,target_image, triangles_number) for i in range(self.population_size)]
        self.target_image = target_image
    def calc_fitnesses(self):
        fitnesses = []
        for chromosome in self.population:
            fitnesses.append(chromosome.fitness())
        return fitnesses
    
    def sort_population(self, fitnesses):
        return [x for _, x in sorted(zip(fitnesses, self.population), key=lambda pair: pair[0])]
    def cross_over(self, parent1, parent2):
        child = Chromosome(self.max_height, self.max_width, self.target_image, len(parent1.triangles))
        split_point = int(random.random()*len(parent1.triangles))
        child.triangles[:split_point] = parent1.triangles[:split_point]
        child.triangles[split_point:] = parent2.triangles[split_point:]
        if child.fitness()> max(parent1.fitness(), parent2.fitness()):
            return child
        return -1
    def cross_over2(self, parent1, parent2):
        child = Chromosome(self.max_height, self.max_width, self.target_image, len(parent1.triangles))
        split_point = int(random.random()*len(parent1.triangles))
        split_point2 = int(random.random()*(len(parent1.triangles)-split_point))
        split_point2= split_point+split_point2
        child.triangles= parent1.triangles[:split_point]+ parent2.triangles[split_point:split_point2]+ parent1.triangles[split_point2:]
        if child.fitness()> max(parent1.fitness(), parent2.fitness()):
            return child
        return -1
        #always return the best choises!
    def mutation(self, parent):
        child = Chromosome(self.max_height, self.max_width, self.target_image, parent.num_triangles)
        child.triangles = [triangle for triangle in parent.triangles]  
        for i in range(len(child.triangles)):
            if random.random() < 0.7:#half of the times
                child.mutate(child.triangles[i])
        if child.fitness() > parent.fitness():
            return child
        return -1
    def tournament_select(self, population, tournament_size):
        indices = np.random.choice(len(population), tournament_size)
        random_subset = [population[i] for i in indices]
        winner = None
        for i in random_subset:
            if winner is None or i.fitness() < winner.fitness():
                winner = i
        return winner

    def run(self, n_generation):
        fitness_history = []
        for iteration in range(n_generation):
            key = False
            fitnesses = self.calc_fitnesses()
            fit_arr = np.array(fitnesses)
            sorted_population = self.sort_population(fitnesses)
            if iteration % 50 == 0:
                best_fitness = np.max(fit_arr)
                fitness_history.append(best_fitness)
                print(f"Generation {iteration}: mean fitness: {np.mean(fit_arr)}, best fitness: {best_fitness}")
                
                

                if iteration % 250 == 0:
                    if len(fitness_history)>=12:
                        if (fitness_history[len(fitness_history)-1])==(fitness_history[len(fitness_history)-11]):
                            key= True
                    if len(fitness_history)==1:
                        best_population = sorted_population[-1]
                        best_image = best_population.draw()
                        best_image.save(f"best_generation_{iteration}.png")
                    elif fitness_history[len(fitness_history)-1]!=fitness_history[len(fitness_history)-6]:
                        best_population = sorted_population[-1]
                        best_image = best_population.draw()
                        best_image.save(f"best_generation_{iteration}.png")
                elif len(fitness_history)>=2:
                    if  fitness_history[len(fitness_history)-1]!=fitness_history[len(fitness_history)-2]:
                        best_population = sorted_population[-1]
                        best_image = best_population.draw()
                        best_image.save(f"best_generation_{iteration}.png")



            new_population = copy.deepcopy(sorted_population[-3:]) 
            while len(new_population) < self.population_size:
                parent_one = self.tournament_select(self.population, int(self.population_size * 0.1))
                parent_two = self.tournament_select(self.population, int(self.population_size * 0.1))
                
                child = 0
                rand = random.uniform(0, 1)
                        
                if rand < 0.6:
                    child = self.cross_over(parent_one, parent_two)
                elif rand >= 0.75:
                    child = self.cross_over2(parent_one, parent_two)
                if 0.4<rand<0.8:
                    child = self.mutation(parent_one)

                if key==True:
                        temp=0
                        print("iteration ",iteration,"rand is", rand,  " in while and child is ", child)
                        so = sorted_population[-2:]
                        while child==-1 and temp<35:
                            turn= 0
                            rand = random.uniform(0, 1)
                            parent_one= None
                            parent_two= None
                            if temp<25:

                                if rand < 0.6:
                                    parent_one = self.tournament_select(self.population, int(self.population_size*0.2))
                                    parent_two = self.tournament_select(self.population, int(self.population_size*0.2))
                                    #temp+= 1
                                    child = self.cross_over(parent_one, parent_two)
                                #if child == None:
                                #   child= parent_one
                                elif rand >= 0.75:
                                    parent_one = self.tournament_select(self.population, int(self.population_size*0.2))
                                    parent_two = self.tournament_select(self.population, int(self.population_size*0.2))
                                    #temp+= 1
                                    child = self.cross_over2(parent_one, parent_two)
                                if 0.3<rand<0.8 and child==-1:
                                    parent_one = self.tournament_select(self.population, int(self.population_size*0.1))
                                    parent_two = parent_one
                                    child = self.mutation(parent_one)
                            
                                print("random is ", rand, " and child is", child, "and parents are ", parent_one.fitness(), " ", parent_two.fitness())
                            if temp>=25:
                                if turn==0:
                                    parent_one= so[1]
                                else: 
                                    parent_one= so[0]
                                turn= 1-turn
                                child= self.mutation(parent_one)
                            temp+=1
                        key=False
                # add child to new population
                if child!=-1:
                    new_population.append(child)
                else:
                    new_population.append(parent_one)

            
            self.population = new_population

    def get_best_of_population(self):
        fitnesses = self.calc_fitnesses()
        sorted_population = [x for _, x in sorted(zip(fitnesses, self.population), key=lambda pair: pair[0])]
        best_population = sorted_population[-1]
        image = best_population.draw()
        cv2.imshow("Reconstructed Image", np.array(image))  
        cv2.waitKey(0)
        cv2.destroyAllWindows()


In [54]:
def resize(image,max_size):
    new_width = int((max_size/max(image.size[0],image.size[1]))* image.size[0])
    new_height = int((max_size/max(image.size[0],image.size[1]))* image.size[1])
    image = image.resize((new_width,new_height), resample=Image.Resampling.LANCZOS)
    return image

Section 5: Result Analysis:
Input image is eagle.jpg
Created images are some snapshots of alternations.

In [55]:
target_image_path = "target_images/eagle.jpg"
image= cv2.imread(target_image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image)
image = resize(image,50)
width,height = image.size
population_size = 25
triangles_number = 25
ngen= 50000
alg = GeneticAlgorithm(width,height,image, population_size,triangles_number)
alg.run(ngen)

Generation 0: mean fitness: -0.2607880784313726, best fitness: -0.18130546837075096
Generation 50: mean fitness: -0.3000529062093445, best fitness: -0.11811789249666989
Generation 100: mean fitness: -0.21247810969362987, best fitness: -0.11811789249666989
Generation 150: mean fitness: -0.21466304300987574, best fitness: -0.11481546103091662
Generation 200: mean fitness: -0.22134466535404015, best fitness: -0.11481546103091662
Generation 250: mean fitness: -0.1852317450553206, best fitness: -0.09521458025079708
Generation 300: mean fitness: -0.20254600867576183, best fitness: -0.09521458025079708
Generation 350: mean fitness: -0.23961085860527614, best fitness: -0.08451149247181543
Generation 400: mean fitness: -0.1782788618208226, best fitness: -0.08451149247181543
Generation 450: mean fitness: -0.14695275660099652, best fitness: -0.08451149247181543
Generation 500: mean fitness: -0.26586227239717436, best fitness: -0.08451149247181543
Generation 550: mean fitness: -0.1630083331741094,

Question 1. State space for each chromosome is any combination of "triangle_numbers" random triangles.
Question 2. 
            Elitism: Directly transfer a percentage of the best chromosomes to the next generation to preserve top solutions and maintain population quality.
            Smart Crossover: Use crossover operators that combine effective segments of parents, such as fitness-based or weighted methods, to create better offspring.
Question 3.
Roulette Wheel Selection: In this method, chromosomes are selected based on their fitness proportion. Each chromosome is given a section of a "roulette wheel" proportional to its fitness. T          he higher the fitness, the larger the section, and thus, the higher the probability of being selected for reproduction. This maintains diversity while giving preference to better solutions
The other is something like tournoment select I have coded.







## Game

In [14]:
import numpy as np
import random
import pygame
import math
from time import sleep, time
ROW_COUNT = 6
COLUMN_COUNT = 7
SQUARESIZE = 100
RADIUS = int(SQUARESIZE / 2 - 5)
PLAYER = 1
CPU = -1
EMPTY = 0
PLAYER_PIECE = 1
CPU_PIECE = -1
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
WINDOW_LENGTH = 4
WINING_SCORE = 10000000000000

In [15]:
class Connect4UI:
    def __init__(self, width=COLUMN_COUNT*SQUARESIZE, height=(ROW_COUNT+1)*SQUARESIZE):
        pygame.init()
        self.width = width
        self.height = height
        self.size = (self.width, self.height)
        self.screen = pygame.display.set_mode(self.size)
        self.font = pygame.font.SysFont("monospace", 75)

    def draw_board(self, board):
        for c in range(COLUMN_COUNT):
            for r in range(ROW_COUNT):
                pygame.draw.rect(self.screen, BLUE, (c * SQUARESIZE, r * SQUARESIZE + SQUARESIZE, SQUARESIZE, SQUARESIZE))
                pygame.draw.circle(self.screen, BLACK, (int(c * SQUARESIZE + SQUARESIZE / 2), int(r * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)

        for c in range(COLUMN_COUNT):
            for r in range(ROW_COUNT):
                if board[r][c] == PLAYER_PIECE:
                    pygame.draw.circle(self.screen, RED, (int(c * SQUARESIZE + SQUARESIZE / 2), self.height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
                elif board[r][c] == CPU_PIECE:
                    pygame.draw.circle(self.screen, YELLOW, (int(c * SQUARESIZE + SQUARESIZE / 2), self.height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)

        pygame.display.update()
        sleep(0.2)


    def display_winner(self, winner):
        if winner == PLAYER:
            label = self.font.render("Player wins!!", 1, RED)
        elif winner == CPU:
            label = self.font.render("Computer wins!!", 1, YELLOW)
        else:
            label = self.font.render("It's a draw!!", 1, BLUE)
        self.screen.blit(label, (40, 10))
        pygame.display.update()
        sleep(5)

MinMax Tree has been implemented.

In [17]:
class Connect4Game:
    def __init__(self, ui, minimax_depth=1, prune=True):
        self.board = np.zeros((ROW_COUNT, COLUMN_COUNT))
        self.ui = Connect4UI() if ui else None
        self.minimax_depth = minimax_depth
        self.prune = prune
        self.current_turn = random.choice([1, -1])

    def drop_piece(self, board, row, col, piece):#to put in some location
        board[row][col] = piece

    def get_next_open_row(self, board,col):#in some col which row is free
        for r in range(ROW_COUNT):
            if board[r][col] == 0:
                return r

    def print_board(self, board):
        print(np.flip(board, 0))

    def winning_move(self, board, piece):
        for c in range(COLUMN_COUNT - 3):
            for r in range(ROW_COUNT):
                if all(board[r][c+i] == piece for i in range(WINDOW_LENGTH)):
                    return True

        for c in range(COLUMN_COUNT):
            for r in range(ROW_COUNT - 3):
                if all(board[r+i][c] == piece for i in range(WINDOW_LENGTH)):
                    return True

        for c in range(COLUMN_COUNT - 3):
            for r in range(ROW_COUNT - 3):
                if all(board[r+i][c+i] == piece for i in range(WINDOW_LENGTH)):
                    return True

        for c in range(COLUMN_COUNT - 3):
            for r in range(3, ROW_COUNT):
                if all(board[r-i][c+i] == piece for i in range(WINDOW_LENGTH)):
                    return True

        return False

    def evaluate_window(self, window, piece):
        score = 0
        opp_piece = PLAYER_PIECE
        if piece == PLAYER_PIECE:
            opp_piece = CPU_PIECE

        if window.count(piece) == 4:
            score += 100
        elif window.count(piece) == 3 and window.count(EMPTY) == 1:
            score += 5
        elif window.count(piece) == 2 and window.count(EMPTY) == 2:
            score += 2
        if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
            score -= 4

        return score

    def score_position(self, board, piece):
        score = 0
        center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
        center_count = center_array.count(piece)
        score += center_count * 3

        for r in range(ROW_COUNT):
            row_array = [int(i) for i in list(board[r,:])]
            for c in range(COLUMN_COUNT-3):
                window = row_array[c:c+WINDOW_LENGTH]
                score += self.evaluate_window(window, piece)

        for c in range(COLUMN_COUNT):
            col_array = [int(i) for i in list(board[:,c])]
            for r in range(ROW_COUNT-3):
                window = col_array[r:r+WINDOW_LENGTH]
                score += self.evaluate_window(window, piece)

        for r in range(ROW_COUNT-3):
            for c in range(COLUMN_COUNT-3):
                window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
                score += self.evaluate_window(window, piece)

        for r in range(ROW_COUNT-3):
            for c in range(COLUMN_COUNT-3):
                window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
                score += self.evaluate_window(window, piece)

        return score

    def is_terminal_node(self, board):#if the game has been finished
        return self.winning_move(board, PLAYER_PIECE) or self.winning_move(board, CPU_PIECE) or len(self.get_valid_locations(board)) == 0

    def get_valid_locations(self, board):#all the locations can be used, which cols arent finished
        valid_locations = []
        for col in range(COLUMN_COUNT):
            if board[ROW_COUNT-1][col] == 0:
                valid_locations.append(col)
        return valid_locations

    def best_cpu_score(self, board, moves):
        move = None
        max_score = -math.inf
        for col in moves:
            row = self.get_next_open_row(board, col)
            b_copy = board.copy()
            self.drop_piece(b_copy, row, col, CPU_PIECE)
            score = self.heuristic(b_copy, CPU_PIECE)
            if score > max_score:
                max_score = score
                move = col
        return move

    def heuristic(self, board, piece):#
        if(self.is_terminal_node(board)):
            if self.winning_move(board, piece):
                return WINING_SCORE
            elif self.winning_move(board, -piece):
                return -WINING_SCORE
            else:
                return 0
        else:
            return self.score_position(board, piece) - self.score_position(board, -piece)

    #TODO: Implement minimax algorithm with alpha-beta pruning and depth limiting
    #self.prune is a boolean that indicates whether to use alpha-beta pruning or not
    #Use the heuristic function as the evaluation function for non terminal nodes
    #Return the column and the score of the best move
    def minimax(self, board, depth, alpha, beta, player):
       # Base cases: if the game is over or depth is 0, return score
        if depth == 0 or self.is_terminal_node(board):
            return None, self.heuristic(board, player)
      
        if player == CPU:
            max_score = -math.inf
            best_move = None
            for col in self.get_valid_locations(board):
                row = self.get_next_open_row(board, col)
                b_copy = board.copy()
                self.drop_piece(b_copy, row, col, CPU_PIECE)
                _, score = self.minimax(b_copy, depth - 1, alpha, beta, PLAYER)
                if score >= max_score:
                    max_score = score
                    best_move = col
                alpha = max(alpha, max_score)
                if beta <= alpha and self.prune:# only if self.prune is activated we have to break
                    break
            return best_move, max_score
        else:
            min_score = math.inf
            best_move = None
            for col in self.get_valid_locations(board):
                row = self.get_next_open_row(board, col)
                b_copy = board.copy()
                self.drop_piece(b_copy, row, col, PLAYER_PIECE)
                _, score = self.minimax(b_copy, depth - 1, alpha, beta, CPU)
                if score <= min_score:
                    min_score = score
                    best_move = col
                beta = min(beta, min_score)
                if beta <= alpha and self.prune:# only if self.prune is activated we have to break
                    break
            return best_move, min_score
 

    def get_cpu_move(self, board, randomness_percent=30):
        moves = self.get_valid_locations(board)
        if len(moves) == 0:
            return None
        random_move = random.choice(moves)
        score_move = self.best_cpu_score(board, moves)
        move = random.choice([score_move] * (100 - randomness_percent) + [random_move] * randomness_percent)
        return move, self.get_next_open_row(board, move)


    def get_human_move(self, board, depth = 1):
        col = self.minimax(board, depth, -math.inf, math.inf, self.current_turn)[0]
        return col, self.get_next_open_row(board, col)

    def play(self):
        winner = None
        while not self.is_terminal_node(self.board):
            if self.ui:
                self.ui.draw_board(self.board)
            if self.current_turn == PLAYER:
                col, row = self.get_human_move(self.board, self.minimax_depth)
                if col is not None:
                    self.drop_piece(self.board, row, col, PLAYER_PIECE)
                    if self.winning_move(self.board, PLAYER_PIECE):
                        winner = PLAYER
                    self.current_turn = CPU
            else:
                col, row = self.get_cpu_move(self.board)
                if col is not None:
                    self.drop_piece(self.board, row, col, CPU_PIECE)
                    if self.winning_move(self.board, CPU_PIECE):
                        winner = CPU
                    self.current_turn = PLAYER

            if self.ui:
                self.ui.draw_board(self.board)
                if winner is not None:
                    self.ui.display_winner(winner)

        if winner is None:
            winner = 0
        return winner

Complete the Connect4Game class below by implementing the `minimax algorithm` with `alpha-beta pruning` and `depth limiting`. Don't change other parts of the code unless you want to define a new heuristic function.

In [18]:
def print_results(results, depth, pruning, start, end):
    pruning_status = "Enabled" if pruning else "Disabled"
    print(
        f'Depth: {depth} | Pruning: {pruning_status} -> User Wins: {results[1]:3}, CPU Wins: {results[-1]:2}, Ties: {results[0]:2}, Time: {end - start:.2f}s'
    )

Use the code below to test the game with the UI. Pygame is not compatible with Jupyter Notebook and will not work in this environment. To use Pygame, please run the code in a separate `.py` file.

In [92]:
game = Connect4Game(True, 3, True)
game.play()

-1

: 

Run the following code to test the results of the game and the time taken to play the game in different `depths` and with `pruning` enabled and disabled.

Results are like this.
When we have the choise of alpha beta pruning, the algorithm uses less time. 
Question 1. As the depth of the algorithm increases, the chances of winning improve due to better evaluation and deeper foresight. Time complexity grows exponentially with depth.
            The number of nodes examined increases with depth. Alpha-beta pruning reduces this number but cannot eliminate the exponential growth entirely.
Question 2. Yes, by sorting the children nodes in some order.
Question 3. The branching factor represents the number of possible moves at any given state. The branching factor is higher at the start and decreases as the board fills up.
Question 4. Alpha-beta pruning skips over branches that cannot influence the final decision, reducing the number of nodes evaluated.
Question 5. Minimax assumes an optimal opponent, but with a random opponent, this assumption is invalid. Your acting method should be appropriate by the conditions.

In [19]:
def check_results():
    for pruning in [True, False]:
        for d in range(1, 4):
            results = {-1: 0, 0: 0, 1: 0}
            start = time()
            for _ in range(20):
                game = Connect4Game(False, d, pruning)
                results[game.play()] += 1
            end = time()
            print_results(results, d, pruning, start, end)

check_results()

Depth: 1 | Pruning: Enabled -> User Wins:  15, CPU Wins:  5, Ties:  0, Time: 0.72s
Depth: 2 | Pruning: Enabled -> User Wins:   1, CPU Wins: 19, Ties:  0, Time: 0.89s
Depth: 3 | Pruning: Enabled -> User Wins:   5, CPU Wins: 15, Ties:  0, Time: 6.57s
Depth: 1 | Pruning: Disabled -> User Wins:  13, CPU Wins:  7, Ties:  0, Time: 0.65s
Depth: 2 | Pruning: Disabled -> User Wins:   0, CPU Wins: 20, Ties:  0, Time: 1.71s
Depth: 3 | Pruning: Disabled -> User Wins:  17, CPU Wins:  3, Ties:  0, Time: 13.29s
